# Which CREs control the cell lineage specific genes

Here, we want to assign the CREs to cell lineage specific genes. We are going to use the regression model from the notebook regressionmodel.ipynb and the assignment of promoters and enhancers per genes.